In [1]:
using Gurobi,JuMP

In [2]:
fix_cost = [20000,20000,10000,30000,20000]
capacity = [700,200,100,200,100]
population = [98000,448000,885000,646000,2719000]

distance = [0 957 1798 165 795;957 0 907 1037 678;1798 907 0 1907 1116;165 1037 1907 0 960;795 678 1116 960 0]
transfer_cost = 0.15*distance
transfer_2 = [287.1,539.4,49.5,238.5,136.05,155.55,101.7,286.05,167.4,144.0]

transfer_cost = 0.15.*distance
transfer_cost[:,1] = 0.3.*distance[:,1]
transfer_cost[1,:] = 0.3.*distance[1,:]
transfer_cost

demand = population * 0.000213

5-element Array{Float64,1}:
  20.874
  95.424
 188.505
 137.598
 579.147

In [20]:
model = Model(solver=GurobiSolver());
n = 5;
@variable(model, a[1:n],Bin)

@variable(model,x[1:n] >=0)

@variable(model,f[1:n] >= 0)

@variable(model,y[1:n,1:n]>=0)

@constraint(model,sum(a[i] for i in 1:5) == 4)

for i in 1:n
    @constraint(model,x[i] <= a[i]*capacity[i])
    @constraint(model,f[i] <= demand[i])
    @constraint(model,x[i] + sum(y[:,i]) == sum(y[i,:])+ f[i])
end

    @objective(model,Max,200*sum(x[i] for i in 1:n) - sum(fix_cost.*a) - sum(transfer_cost.*y))

model;

In [21]:
solve(model);

est_x = getvalue(x);
est_f = getvalue(f);
est_y = getvalue(y);
est_a = getvalue(a);

est_x;

est_y;

est_a;

est_f;

getobjectivevalue(model)

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Optimize a model with 16 rows, 40 columns and 69 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 7e+02]
  Objective range [5e+01, 3e+04]
  Bounds range    [1e+00, 1e+00]
  RHS range       [4e+00, 6e+02]
Found heuristic solution: objective -70000
Presolve removed 10 rows and 10 columns
Presolve time: 0.00s
Presolved: 6 rows, 30 columns, 51 nonzeros
Variable types: 29 continuous, 1 integer (1 binary)

Root relaxation: objective 7.332631e+03, 5 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    7332.6313000 7332.63130  0.00%     -    0s

Explored 0 nodes (5 simplex iterations) in 0.00 seconds
Thread count was 4 (of 4 available processors)

Optimal solution found (tolerance 1.00e-04)
Best objective 7.332631300000e+03, best bound 7.332631300000e+03

7332.631300000023

In [22]:
result = [est_x est_f est_a]

5×3 Array{Float64,2}:
 633.043   20.874   1.0
 200.0     95.424   1.0
 100.0    100.0     1.0
   0.0    137.598  -0.0
   0.0    579.147   1.0

In [23]:
est_y

5×5 Array{Float64,2}:
 0.0  0.0  0.0  612.169    0.0  
 0.0  0.0  0.0    0.0    104.576
 0.0  0.0  0.0    0.0      0.0  
 0.0  0.0  0.0    0.0    474.571
 0.0  0.0  0.0    0.0      0.0  

In [52]:
capacity = [700,200,100,200,0]
model = Model(solver=GurobiSolver());
n = 5;
@variable(model, a[1:n],Bin)

@variable(model,x[1:n] >=0)

@variable(model,f[1:n] >= 0)

@variable(model,y[1:n,1:n]>=0)

for i in 1:n
    @constraint(model,x[i] <= a[i]*capacity[i])
    @constraint(model,f[i] <= demand[i])
    @constraint(model,x[i] + sum(y[:,i]) == sum(y[i,:])+ f[i])
end

@objective(model,Max,200*sum(x[i] for i in 1:n) - 100000 - sum(transfer_cost.*y))

model;

In [53]:
solve(model);
est_x = getvalue(x);
est_f = getvalue(f);
est_y = getvalue(y);
est_a = getvalue(a);

est_x;
est_y;
est_a;
est_f;
getobjectivevalue(model)

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Optimize a model with 15 rows, 40 columns and 64 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 7e+02]
  Objective range [5e+01, 5e+02]
  Bounds range    [1e+00, 1e+00]
  RHS range       [2e+01, 6e+02]
Found heuristic solution: objective -0
Presolve removed 10 rows and 11 columns
Presolve time: 0.00s
Presolved: 5 rows, 29 columns, 49 nonzeros
Variable types: 29 continuous, 0 integer (0 binary)

Root relaxation: objective 8.723263e+04, 4 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    87232.631300 87232.6313  0.00%     -    0s

Explored 0 nodes (4 simplex iterations) in 0.00 seconds
Thread count was 4 (of 4 available processors)

Optimal solution found (tolerance 1.00e-04)
Best objective 8.723263130000e+04, best bound 8.723263130000e+04, ga

-12767.368699999963

In [38]:
result = [est_x est_f est_a]

5×3 Array{Float64,2}:
 433.043   20.874  1.0
 200.0     95.424  1.0
   0.0      0.0    0.0
 200.0    137.598  1.0
   0.0    579.147  0.0

In [34]:
sum(est_x)

837.6190000000003

In [27]:
est_x[1] + sum(est_y[:,1]) == sum(est_y[1,:])+ est_f[1]

true

In [29]:
sum(est_y[1,:])

412.169

In [39]:
est_y

5×5 Array{Float64,2}:
 0.0  0.0  0.0  412.169    0.0  
 0.0  0.0  0.0    0.0    104.576
 0.0  0.0  0.0    0.0      0.0  
 0.0  0.0  0.0    0.0    474.571
 0.0  0.0  0.0    0.0      0.0  

In [19]:
transfer_cost

5×5 Array{Float64,2}:
   0.0  287.1   539.4    49.5   238.5
 287.1    0.0   136.05  155.55  101.7
 539.4  136.05    0.0   286.05  167.4
  49.5  155.55  286.05    0.0   144.0
 238.5  101.7   167.4   144.0     0.0